In [1]:
import iris
import os
import shutil
import dask

In [2]:
SAMPLE_DATA_PATH="/data/cssp-china/sample-data-17-01-20/cssp_china_pp"

In [3]:
files = [os.path.join(SAMPLE_DATA_PATH, f) for f in os.listdir(SAMPLE_DATA_PATH)]

In [4]:
%%time
cubes = iris.load(files[2])

/envs/auto-build-envs/datasci/lib/python3.6/site-packages/iris/analysis/_interpolation.py:300: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data = data[dim_slices]


CPU times: user 44.8 s, sys: 4.04 s, total: 48.9 s
Wall time: 48.7 s


In [5]:
cubes

[<iris 'Cube' of m01s01i204 / (unknown) (time: 10; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of m01s03i254 / (unknown) (time: 10; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of m01s03i255 / (unknown) (time: 10; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of m01s03i287 / (unknown) (pseudo_level: 9; time: 10; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of m01s03i288 / (unknown) (pseudo_level: 9; time: 10; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of m01s03i289 / (unknown) (pseudo_level: 5; time: 10; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of m01s03i290 / (unknown) (pseudo_level: 9; time: 10; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of m01s03i291 / (unknown) (pseudo_level: 5; time: 10; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of m01s03i292 / (unknown) (pseudo_level: 5; time: 10; grid_latitude: 219; grid_longitude: 286)>,
<iris 'Cube' of Evaporation from soil surface / (kg/m^2/s) (

In [6]:
cubes[-4]

<iris 'Cube' of x_wind / (m s-1) (time: 10; grid_latitude: 218; grid_longitude: 286)>

In [7]:
# c = [c for c in cubes if 'air_temperature' in c.name()][3]

In [8]:
# c

In [9]:
PARAM_NAME = 'x_wind_10m'
PARAM_CONSTRAINT = iris.AttributeConstraint(STASH='m01s03i225')

In [10]:
from dask import bag as db

In [11]:
from collections import namedtuple

Paths = namedtuple('Paths',['old','new'])
def get_new_and_old_path(path, param_name):
    filename = os.path.basename(path)
    new_filename = filename[:-2] + 'nc' if filename[-2:].lower() == 'pp' else filename + '.nc'
    new_filename = f"{param_name}_{new_filename}"
    old_dir = os.path.dirname(path)
    new_dir = old_dir[:-2] + 'nc' if old_dir[:-2].lower() == 'pp' else old_dir + '_nc'
    new_path = os.path.join(new_dir, param_name, new_filename)
    return Paths(path, new_path)

eg_path = get_new_and_old_path(files[0],PARAM_NAME )
eg_path

Paths(old='/data/cssp-china/sample-data-17-01-20/cssp_china_pp/apepda.pae16i0.pp', new='/data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/x_wind_10m/x_wind_10m_apepda.pae16i0.nc')

In [12]:
def not_converted(paths):
    return not os.path.exists(paths.new)

not_converted(eg_path)

True

In [13]:
paths = eg_path

In [14]:
paths

Paths(old='/data/cssp-china/sample-data-17-01-20/cssp_china_pp/apepda.pae16i0.pp', new='/data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/x_wind_10m/x_wind_10m_apepda.pae16i0.nc')

In [15]:
# import shutil
# import dask
# def local_convert(paths):
#     tmp_path = '/tmp/'+os.path.basename(paths.old)
#     tmp_path_new = '/tmp/'+os.path.basename(paths.new)
#     shutil.copy(paths.old, tmp_path)
    
#     with dask.config.set(scheduler='single-threaded'):
#         precipitation_flux
#         iris.fileformats.netcdf.save(cube, tmp_path_new, zlib=True, complevel=2)
    
    
#     shutil.copy(tmp_path_new, paths.new)
    
#     try:
#         os.remove(tmp_path_new)
#     except Exception:
#         print(f"could not delete {tmp_path_new}")
              
#     try:
#         os.remove(tmp_path)
#     except Exception:
#         print(f"could not delete {tmp_path}")

#     return paths

In [16]:
from pathlib import Path
def convert(paths, param_constraint):
    with dask.config.set(scheduler='single-threaded'):
        cubes = iris.load(paths.old, param_constraint)
        Path(os.path.dirname(paths.new)).mkdir(parents=True, exist_ok=True)
        iris.fileformats.netcdf.save(cubes, paths.new, zlib=True, complevel=2)
        return paths

In [17]:
# %%time
# convert(eg_path, PARAM_CONSTRAINT)

In [18]:
# %%time
# local_convert(eg_path)PARAM_CONSTRAINT

In [1]:
import distributed
import dask
from dask_kubernetes import KubeCluster
from dask import array as da

from dask.diagnostics import ProgressBar

from functools import partial



In [20]:
conversion = db.from_sequence(files).map(partial(get_new_and_old_path, param_name=PARAM_NAME)).filter(not_converted).map(partial(convert, param_constraint=PARAM_CONSTRAINT))
conversion

dask.bag<convert..., npartitions=116>

In [2]:
cluster = KubeCluster()
cluster.scale_up(100)
display(cluster)
# Connect dask to the cluster
client = distributed.Client(cluster)
client

distributed.dashboard.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://10.244.0.203:44423
distributed.scheduler - INFO -   dashboard at:                     :8787


distributed.scheduler - INFO - Receive client connection: Client-2d176e88-49b0-11ea-8082-c3f0ab157203
distributed.core - INFO - Starting established connection


Client Scheduler: tcp://10.244.0.203:44423 Dashboard: /user/tam203/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


distributed.scheduler - INFO - Register tcp://10.244.0.208:38579
distributed.scheduler - INFO - Register tcp://10.244.0.210:38661
distributed.scheduler - INFO - Register tcp://10.244.0.211:36297
distributed.scheduler - INFO - Register tcp://10.244.0.206:40251
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.208:38579
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.244.0.207:39875
distributed.scheduler - INFO - Register tcp://10.244.0.209:40423
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.210:38661
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.211:36297
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.244.0.206:40251
distributed.core - INFO - Starting established connection
distributed.scheduler - IN

In [22]:
%%time
with ProgressBar():
    converted = conversion.compute()
converted

CPU times: user 33.2 s, sys: 3.8 s, total: 37 s
Wall time: 5min 38s


[Paths(old='/data/cssp-china/sample-data-17-01-20/cssp_china_pp/apepda.pae16i0.pp', new='/data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/x_wind_10m/x_wind_10m_apepda.pae16i0.nc'),
 Paths(old='/data/cssp-china/sample-data-17-01-20/cssp_china_pp/apepda.pac83i0.pp', new='/data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/x_wind_10m/x_wind_10m_apepda.pac83i0.nc'),
 Paths(old='/data/cssp-china/sample-data-17-01-20/cssp_china_pp/apepda.pae17i0.pp', new='/data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/x_wind_10m/x_wind_10m_apepda.pae17i0.nc'),
 Paths(old='/data/cssp-china/sample-data-17-01-20/cssp_china_pp/apepda.pac81i0.pp', new='/data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/x_wind_10m/x_wind_10m_apepda.pac81i0.nc'),
 Paths(old='/data/cssp-china/sample-data-17-01-20/cssp_china_pp/apepda.pa508i0.pp', new='/data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/x_wind_10m/x_wind_10m_apepda.pa508i0.nc'),
 Paths(old='/data/cssp-china/sample-data-17-01-20/cssp_china_pp/a

In [ ]:
!ls /data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/precipitation_flux/

In [ ]:
!ls -c data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/y_wind_10m/

In [ ]:
!ls /data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/

In [23]:

!ls /data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/y_wind_10m/ | wc -l
!ls /data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/precipitation_flux| wc -l
!ls /data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/air_temperature_mean_at_pressure_levels| wc -l
!ls /data/cssp-china/sample-data-17-01-20/cssp_china_pp_nc/x_wind_10m| wc -l


232
232
232
232
